### Working with MOD13 Vegetation Indices data

I assume this process is going to be similar to burn data? 

In [24]:
# Import libraries
import os
import glob
from osgeo import gdal
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import scipy.ndimage
import pandas as pd
import datetime as dt
import rasterio as rio
import re
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em

In [25]:
# Set input directory, and change working directory
inDir = "C:\\Users\\samgr\\OneDrive\\Desktop\\Masters_Project\\Preparation"   # IMPORTANT: Update to reflect directory on your OS
os.chdir(inDir)                                                               # Change to working directory
outDir = os.path.normpath(os.path.split(inDir)[0] + os.sep + 'output') + '\\' # Create and set output directory
if not os.path.exists(outDir): os.makedirs(outDir)

In [26]:
#haven't got MODIS downloaded yet
MODISFiles = glob.glob('MOD13A3**.hdf') # Search for and create a list of EVI files
MODISFiles

['MOD13A3.A2015001.h19v07.006.2015295114815.hdf',
 'MOD13A3.A2015032.h19v08.006.2015297000353.hdf',
 'MOD13A3.A2015060.h19v08.006.2015298212508.hdf',
 'MOD13A3.A2015091.h19v08.006.2015299041123.hdf',
 'MOD13A3.A2015152.h20v08.006.2015301214710.hdf',
 'MOD13A3.A2015274.h19v07.006.2015317210610.hdf',
 'MOD13A3.A2015305.h19v07.006.2015343135029.hdf',
 'MOD13A3.A2015305.h19v08.006.2015343135001.hdf',
 'MOD13A3.A2015305.h20v08.006.2015343135325.hdf',
 'MOD13A3.A2015335.h19v07.006.2016007181745.hdf',
 'MOD13A3.A2015335.h19v08.006.2016007180603.hdf',
 'MOD13A3.A2015335.h20v08.006.2016007181735.hdf']

In [27]:
print(MODISFiles[0])

MOD13A3.A2015001.h19v07.006.2015295114815.hdf


In [37]:
# File name metadata:
productId = MODISFiles[0].split('.')[0]                                          # First: product name
yeardoy = MODISFiles[0].split(productId + '.A')[1].split('.')[0]                 #julian date
date = dt.datetime.strptime(yeardoy, '%Y%j').strftime('%d/%m/%Y')              # Convert YYYYDDD to DD/MM/YYYY
tiles_id = MODISFiles[0].split(yeardoy + '.')[1].split('.006')[0]               # Second: layer name
unknown_format = MODISFiles[0].split(tiles_id +'.006.')[1].split('.hdf')[0]                  # Third: date


print('Product Name: {}\nLayer ID: {}\nDate of Observation: {}\n(Not sure what this represents: {})'.format(productId, tiles_id, date, unknown_format ))

Product Name: MOD13A3
Layer ID: h19v07
Date of Observation: 01/01/2015
(Not sure what this represents: 2015295114815)


Find out what last part of naming convention represents

In [30]:
# View dataset metadata
with rio.open(MODISFiles[0]) as dataset:
    print(dataset)
    hdf4_meta = dataset.meta

# Notice that there are metadata at the highest level of the file
hdf4_meta

<open DatasetReader name='MOD13A3.A2015001.h19v07.006.2015295114815.hdf' mode='r'>


C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


{'driver': 'HDF4',
 'dtype': 'float_',
 'nodata': None,
 'width': 512,
 'height': 512,
 'count': 0,
 'crs': None,
 'transform': Affine(1.0, 0.0, 0.0,
        0.0, 1.0, 0.0)}

In [32]:
# Print all of the subdatasets in the data
with rio.open(MODISFiles[0]) as dataset:
    crs = dataset.read_crs()
    for name in dataset.subdatasets:
        print(name)

HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly NDVI
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly relative azimuth angle
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly pixel reliability
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly EVI
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly VI Quality
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly red reflectance
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly NIR reflectance
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly blue reflectance
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1